In [1]:
import networkx as nx
import numpy as np
import pandas as pd

In [2]:
graph_path = "graph.net"
edges = [line.strip().split() for line in open(graph_path)]
edges[:10]

[['1', '88160'],
 ['1', '118052'],
 ['1', '161555'],
 ['1', '244916'],
 ['1', '346495'],
 ['1', '444232'],
 ['1', '447165'],
 ['1', '500600'],
 ['2', '27133'],
 ['2', '62291']]

In [3]:
edges_num = []
for e in edges:
    edges_num.append([int(e[0]), int(e[1])])

In [4]:
G = nx.Graph()
G.add_edges_from(edges_num)

In [5]:
import community as community_louvain

In [6]:
partition = community_louvain.best_partition(G)

In [7]:
num_communities = len(set(partition.values()))
num_communities

231

In [8]:
modularity = community_louvain.modularity(partition, G)
modularity

0.9261968232047458

In [9]:
max(partition.values())

230

In [10]:
communities = {}
for key, value in partition.items():
    if value in communities:
        communities[value].append(key)
    else:
        communities[value] = [key]

# Number of blocks (communities)
num_blocks = max(partition.values()) + 1

# Initialize edge-propensity parameters
theta = np.ones((num_blocks, num_blocks))

# EM algorithm
def em_algorithm(G, partition, num_blocks, theta_init=None):
    if theta_init is None:
        theta = np.ones((num_blocks, num_blocks))
    else:
        theta = theta_init.copy()
    
    for _ in range(100): # Maximum number of iterations
        # E-step: Estimate the community assignments
        for u, v in G.edges():
            r = partition[u]
            s = partition[v]
            theta[r, s] += 1
        
        # M-step: Update the edge-propensity parameters
        for i in range(num_blocks):
            for j in range(num_blocks):
                if theta[i, j] > 0:
                    theta[i, j] /= np.sum(theta[i, :])
    
    return theta

optimal_theta = em_algorithm(G, partition, num_blocks)

optimal_theta.shape

purchase_probabilities = {}
for u, v in G.edges():
    r = partition[u]
    s = partition[v]
    theta_rs = optimal_theta[r, s]
    c = G.degree(u) # Degree of customer node
    p = G.degree(v) # Degree of product node
    # Scaling factor (this is an example; adjust as needed)
    scaling_factor = 1.0 / (c * p)
    probability = theta_rs * scaling_factor
    purchase_probabilities[(u, v)] = probability

list(purchase_probabilities.keys())[:2]

In [11]:
list_items = []
with open('item_sets.txt') as f:
    for line in f:
        list_items.append(list(map(int, line.strip().split())))
list_items[0]

[100005, 127545, 202036, 257630, 362970, 376927]

In [12]:
def GetGlobalCommunity(partition, items):
    d = {}
    for item in items:
        if partition[item] in d:
            d[partition[item]] += 1
        else:
            d[partition[item]] = 1
    max_key = max(d, key=d.get)
    return max_key

In [13]:
GetGlobalCommunity(partition, list_items[0])

19

In [14]:
def GetItems(G, items):
    d = {}
    for item in items:
        neighbors = G.neighbors(item)
        for neighbor in neighbors:
            if neighbor not in items:
                if neighbor in d:
                    d[neighbor] += 1
                else:
                    d[neighbor] = 1
    keys_with_height_value = [key for key, value in d.items() if value == max(list(d.values()))]
    
    return keys_with_height_value

In [15]:
GetItems(G, list_items[0])

[237164, 262654, 429080]

In [16]:
def PredictItem(G, partition, communities,items):

    comm = GetGlobalCommunity(partition, items)
    cand = [c for c in communities[comm] if c > max(items)]
    min_path = -1
    if cand == []:
        best_item = comm[0]
    else:
        best_item = cand[0]
    
    for c in cand:
        shortest_paths = []
        for item in items:
            try:
                shortest_path_length = nx.shortest_path_length(G, c, item)
                shortest_paths.append(shortest_path_length)
            except nx.NetworkXNoPath:
                pass
        if shortest_paths:
            avg_path = sum(shortest_paths) / len(shortest_paths)
            if avg_path < min_path or min_path == -1:
                min_path = avg_path
                best_item = c
    return best_item

In [17]:
PredictItem(G, partition, communities, list_items[0])

429080

In [18]:
from tqdm import tqdm

In [31]:
result = []
for i in tqdm(range(297)):
    result.append(PredictItem(G, partition, communities, list_items[i]))

100%|█████████████████████████████████████████| 297/297 [19:11<00:00,  3.88s/it]


In [23]:
len(result)

297

In [24]:
result

[429080,
 540746,
 540746,
 315862,
 464375,
 473459,
 548331,
 411257,
 519950,
 439913,
 347354,
 347354,
 461392,
 461392,
 419049,
 495234,
 536191,
 536191,
 536191,
 501457,
 422589,
 437373,
 437373,
 479613,
 461186,
 543168,
 444737,
 444737,
 378414,
 536726,
 405763,
 394213,
 100111,
 494944,
 352050,
 352050,
 515230,
 522785,
 465679,
 159595,
 435490,
 283815,
 259615,
 535160,
 522190,
 438740,
 480312,
 495760,
 501895,
 369070,
 522837,
 495641,
 539204,
 532084,
 323322,
 503504,
 534769,
 534769,
 534769,
 534769,
 534769,
 353292,
 534769,
 467284,
 378651,
 537030,
 495912,
 548471,
 548471,
 543922,
 377811,
 234710,
 189332,
 272915,
 529747,
 483498,
 529747,
 529747,
 259819,
 259819,
 546290,
 414884,
 414884,
 495018,
 542541,
 546290,
 539828,
 283343,
 548002,
 519723,
 519723,
 544392,
 546414,
 478426,
 457975,
 438065,
 545871,
 539828,
 548425,
 545006,
 545186,
 545006,
 545006,
 545356,
 548425,
 540951,
 499553,
 453047,
 119051,
 373177,
 535968,
 

In [25]:
new_result = result + [0] * (len(list_items) - len(result))
new_result

[429080,
 540746,
 540746,
 315862,
 464375,
 473459,
 548331,
 411257,
 519950,
 439913,
 347354,
 347354,
 461392,
 461392,
 419049,
 495234,
 536191,
 536191,
 536191,
 501457,
 422589,
 437373,
 437373,
 479613,
 461186,
 543168,
 444737,
 444737,
 378414,
 536726,
 405763,
 394213,
 100111,
 494944,
 352050,
 352050,
 515230,
 522785,
 465679,
 159595,
 435490,
 283815,
 259615,
 535160,
 522190,
 438740,
 480312,
 495760,
 501895,
 369070,
 522837,
 495641,
 539204,
 532084,
 323322,
 503504,
 534769,
 534769,
 534769,
 534769,
 534769,
 353292,
 534769,
 467284,
 378651,
 537030,
 495912,
 548471,
 548471,
 543922,
 377811,
 234710,
 189332,
 272915,
 529747,
 483498,
 529747,
 529747,
 259819,
 259819,
 546290,
 414884,
 414884,
 495018,
 542541,
 546290,
 539828,
 283343,
 548002,
 519723,
 519723,
 544392,
 546414,
 478426,
 457975,
 438065,
 545871,
 539828,
 548425,
 545006,
 545186,
 545006,
 545006,
 545356,
 548425,
 540951,
 499553,
 453047,
 119051,
 373177,
 535968,
 

In [26]:
d_fin = {'id': list(range(1, len(list_items)+1)), 'target': new_result}
df = pd.DataFrame(d_fin)
df.head()

,id,target
0,1,429080
1,2,540746
2,3,540746
3,4,315862
4,5,464375


In [27]:
df.to_csv("submission_late_02.csv", index=False)

In [28]:
len(list_items)

75149

In [29]:
len(result)

297

In [30]:
correct = (0.0021 * 75149) / 297
correct

0.5313565656565656